# HaloFerax Recombination Project Illustration (simulated data).

### Housekeeping
Our first step is to import some tools that we will need for our anaylsis.

There are a lot of things here and we will talk our way through some of them.

In [ ]:
#This hides some warnings that we might want to look at one day if our code doesn't work!
import warnings
warnings.filterwarnings('ignore')

#These are various graph plotting and data processing tools we may use.
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
import numpy as np
import pandas as pd

#This is a nice plotting library that will also do some pretty graphics for us.
import aplanat
from aplanat import points
from aplanat import graphics
from aplanat.hist import histogram
from bokeh.layouts import gridplot

import pysam

from aplanat.lines import steps

## Alignment
We will first need to compare our sequenced reads with the reference genome to determine which reads are like which reference. We will do this in a few different ways to see how it is possible.

We need to define the reference sequence we are going to map our data to. In this case it will be a merged reference for our two reference genomes.

We will have a look at these files in the terminal.

In [ ]:
reference = "~/student_projects_2022/data/refs/merged_refs.fasta"

We next need to find our sequence reads that we are going to map. These will be all compressed as fastq.gz files.

A fastq file consists of four lines for each "read".

These are:

\@ReadName
ATGC
.
!@£&

Where the first line is the read name.
The second line is the read sequence.
The third line is a divider.
The fourth line is a quality score (encoded in ascii).


In [ ]:
reads = "~/student_projects_2022/data/reads/run1/*.fastq.gz"

Now we are going to map our reads against the reference using a program called minimap2. You can find out more about minimap2 here: https://github.com/lh3/minimap2

The paper describing minimap2 is available here:
https://pubmed.ncbi.nlm.nih.gov/29750242/

In [ ]:
!minimap2 -ax map-ont -t 4 $reference $reads > reads_reference.sam


We've now generated a sam file - SAM Sequence Alignment Map.

These files are big and messy - we want to sort the file so we can look at our data. We also want to store it efficiently. So we are going to convert it to a bam file - this is a Binary sequence Aligment File.

To do this, we will use a program called SAMTOOLS:

See - http://www.htslib.org/


In [ ]:
!samtools sort -o sorted.reads_reference.bam reads_reference.sam

The problem with a bam file is it is hard to look at - so we want to have an index that allows us to find what we need.

In [ ]:
!samtools index sorted.reads_reference.bam

OK - Great! Now - what do these files look like?
To answer this we need to use another feature of samtools - samtools view.

In [ ]:
# view the first 10 lines of the bam file
!samtools view -h sorted.reads_reference.bam 2> /dev/null | head -n 10

OK - that's not very easy to understand. Can we get some more information on what is happening here?

What is a sam file?

Its a way of storing alignments to a reference. 

Can we get more information from it?


In [ ]:
# count the alignments in a file
!samtools view -bc sorted.reads_reference.bam

We can get more detailed information on this file by looking at the statistics for reads contained within it.

In [ ]:
!samtools flagstat sorted.reads_reference.bam

We can analyse the file in more detail by generating some more advanced statistics using a program called stats_from_bam which comes from a package called pomoxis.

In [ ]:
# run the alignment summarizer program
!stats_from_bam sorted.reads_reference.bam > sorted.reads_reference.bam.stats

# use python to plot the results
import pandas as pd
import aplanat
from aplanat.hist import histogram
from bokeh.layouts import gridplot
df = pd.read_csv("sorted.reads_reference.bam.stats", sep="\t")

p1 = histogram(
    [df['read_length']], title="Read lengths",
    x_axis_label="read length / bases", y_axis_label="count")
p1.xaxis.formatter.use_scientific = False
p2 = histogram(
    [df['acc']], title="Read accuracy",
    x_axis_label="% accuracy", y_axis_label="count")
aplanat.show(gridplot((p1, p2), ncols=2))

We can use the data about this run to calculate some additional statistics.
We can look at the data we have.

In [ ]:
df

In [ ]:
from aplanat import graphics

summary = graphics.InfoGraphItems()
summary.append(label='Total reads', value=len(df.name.unique()), icon='angle-up', unit='')
summary.append('Total yield', df.read_length.sum(), 'signal', 'b')
summary.append('Mean read length', df.read_length.sum()/len(df.name.unique()), 'align-center', 'b')
summary.append('Mean read identity', df.iden.mean(), 'thumbs-up')
summary.append('Mean read accuracy', df.acc.mean(), 'thumbs-down')
plot = graphics.infographic(summary.values())
aplanat.show(plot, background='#f4f4f4')

These plots and figures are for simluated data - the are not the same as you will see for your real data!

## Plotting depth of coverage across a BAM file with Mosdepth

A very common requirement is to investigate the depth of coverage across the genome, i.e. the number of reads on average overlapping any given locus. This may be in order to determine how well a sequencing run performed in terms of on-target throughput, to check if we have enough depth of coverage to support variant calling, or to find out if there are any areas of the genome that have been under-represented in our sequencing run.

Whilst samtools does provide tools for performing these checks, mosdepth is simpler to use whilst also being faster, making it a natural choice for running coverage analysis.

Fundamentally mosdepth accepts a BAM file and in turn creates several output files containing coverage information. Before we look at those, let's go over the command we'll run:

mosdepth is the base command,
-n means don't output per base coverage information, as it is unneeded, instead mosdepth will output coverage by intervals,
--by sets the interval size,
--fast-mode makes some performance optimizations,
test_cov is the name we'll give to mosdepth to prefix our output files with.
sorted.reads_reference.bam is our trusty BAM which will be analysed.

In [ ]:
!mosdepth -n --fast-mode --by 100 test_cov sorted.reads_reference.bam

## Plotting Coverage

To plot the coverage we are going to use some python code to visualise everything. Yes this is a bit complex - but you don't need all the details.

In [ ]:
cumulative_depth = pd.read_csv(
    'test_cov.mosdepth.region.dist.txt', sep='\t',
    names=['ref', 'depth', 'proportion'])

binned_depth = pd.read_csv(
    'test_cov.regions.bed.gz', sep='\t',
    names=['ref', 'start', 'end', 'depth'])

def make_coverage_plot(cumulative_depth, binned_depth):
    # Plot the proportion of the genome at coverage levels
    p1 = steps(
        [cumulative_depth[cumulative_depth['ref'].eq(binned_depth['ref'].unique()[0])]['depth']],
        [cumulative_depth[cumulative_depth['ref'].eq(binned_depth['ref'].unique()[0])]['proportion']],
        colors=['darkolivegreen'],
        x_axis_label='Depth of coverage',
        y_axis_label='Proportion of genome at coverage',
        title=binned_depth['ref'].unique()[0])
    
    # Plot the binned coverage levels across the genome
    
    p2 = steps(
        [binned_depth[binned_depth['ref'].eq(binned_depth['ref'].unique()[0])]['start']],
        [binned_depth[binned_depth['ref'].eq(binned_depth['ref'].unique()[0])]['depth']],
        colors=['darkolivegreen'],
        x_axis_label='Position along reference',
        y_axis_label='sequencing depth / bases',
        title=binned_depth['ref'].unique()[0])
    p2.xaxis.formatter.use_scientific = False
    
    p3 = steps(
        [cumulative_depth[cumulative_depth['ref'].eq(binned_depth['ref'].unique()[1])]['depth']],
        [cumulative_depth[cumulative_depth['ref'].eq(binned_depth['ref'].unique()[1])]['proportion']],
        colors=['darkblue'],
        x_axis_label='Depth of coverage',
        y_axis_label='Proportion of genome at coverage',
        title=binned_depth['ref'].unique()[1])

    
    # Plot the binned coverage levels across the genome
    
    p4 = steps(
        [binned_depth[binned_depth['ref'].eq(binned_depth['ref'].unique()[1])]['start']],
        [binned_depth[binned_depth['ref'].eq(binned_depth['ref'].unique()[1])]['depth']],
        colors=['darkblue'],
        x_axis_label='Position along reference',
        y_axis_label='sequencing depth / bases',
        title=binned_depth['ref'].unique()[1])
    p4.xaxis.formatter.use_scientific = False
    return gridplot((p1, p2,p3,p4), ncols=2)

aplanat.show(make_coverage_plot(cumulative_depth, binned_depth), background="#f4f4f4")

Hold on!

The observant amongst you will realise that this tells tells us roughly where our recombinants are! But we want to examine this in more detail. You can use the interactive features of the plots above to zoom in and get a better idea of where the coverage profiles switch. But we really want a higher resolution plot of some kind.


We could do this by changing the mosdepth parameters above (see the --by value). Perhaps play with that a bit and see if you can get better resolution.

But there is another way we can look.

In [ ]:
from igv_jupyterlab import IGV
import os
user = os.getenv('JUPYTERHUB_USER')

url=f"http://10.157.200.14/user/{user}/tree/UnderGradProjectTest/"
bams={'results':'sorted.reads_reference.bam'}
track_list=[
            {"name": "HMerge",
                "url": url+"data/refs/merge.gff3",
                "format": "gff3",
                "type": "annotation",
                "displayMode": "expanded",
                "height":120,
                "indexed": False }
]
colors=['orange','green','gray']
i=0
for b in bams:
    d = {"name": b,
        "url":url+bams[b],
        "type": "alignment",
         "displayMode":"SQUISHED",
         "height":400,
         "removable":True,
         #"color":colors[i],
        "indexed": True }
    track_list.append(d)
    i+=1
    
genome = IGV.create_genome(
    name="merged_refs",   
    fasta_url=url+'data/refs/merged_refs.fasta',
    index_url=url+ 'data/refs/merged_refs.fasta.fai',
    tracks=track_list
)

#create the widget
igv = IGV(genome=genome)


display(igv)

## Explore the data

Using the plots above you should be able to find the approximate coordinates where the reads switch from one reference to another.

In [ ]:

bams={'results':'sorted.reads_reference.bam'}
track_list=[
            {"name": "HMerge",
                "url": url+"data/refs/merge.gff3",
                "format": "gff3",
                #"type": "annotation",
                "displayMode": "expanded",
                "height":120,
                "indexed": False }
]
colors=['orange','green','gray']
i=0
for b in bams:
    d = {"name": b,
        "url":url+bams[b],
        "type": "alignment",
         "displayMode":"SQUISHED",
         "height":500,
         "removable":True,
         #"color":colors[i],
        "indexed": True }
    track_list.append(d)
    i+=1
    
genome2 = IGV.create_genome(
    name="custom_reference",   
    fasta_url=url+'data/refs/merged_refs.fasta',
    index_url=url+ 'data/refs/merged_refs.fasta.fai',
    tracks=track_list
)

#create the widget
igv2 = IGV(genome=genome2)
igv2.locus = "NC_013967.1:1-10000"
#igv.search_locus('NC_013967.1',1,100000)
display(igv2)

## What about assembly?

Assembly is the process of generating a sequence from our reads - we can use a tool called "Flye" to do this for us - see https://github.com/fenderglass/Flye for info.

In [ ]:
!flye

In [ ]:
!flye --threads 4 --out-dir assembly --nano-hq $reads

This assembly gives us some information that might be useful to us. Have a look at the assembly statistics. What do they mean?


We can use a tool called last (made up of programs including lastdb, lastal and last-dotplot) to have a look at our assembly compared with the reference genomes.

In [ ]:
!lastdb merge_refdb $reference

In [ ]:
!lastal --split merge_refdb assembly/assembly.fasta > test.maf


In [ ]:
!lastal -h


In [ ]:
!last-dotplot  -v test.maf test.png

In [ ]:
!last-dotplot -h


In [ ]:
from IPython.display import Image
Image("test.png")

## Using last to look at read mappings.

Can we use the last tool to look at our read mappings in more detail?

This is a bit problematic because last requires a fasta file and we have all our reads as fastq files.

So we need to convert all our fastq into fasta.

How?


In [ ]:
print (reads)

The zcat command will combine all our fastq files into one big fastq file.

In [ ]:
!zcat $reads > allreads.fastq

Now we use a special command called "sed" to edit our file.

In [ ]:
!sed -n '1~4s/^@/>/p;2~4p'  allreads.fastq > allreads.fasta

In [ ]:
!lastal --split -m 1 merge_refdb allreads.fasta > reads_last.maf

Now we are going to use a program called dnarrange to find all the reads which map to both references.

You can read about dnarrange here - https://github.com/mcfrith/dnarrange

In [ ]:
!dnarrange reads_last.maf > groups0.maf

In [ ]:
!dnarrange -h


In [ ]:
!head -100 groups0.maf

In [ ]:
!dnarrange-link groups0.maf > linked.txt

In [ ]:
!head linked.txt


These coordinates (above) should give us the approximate regions to look at in the IGV outputs to find our recombinants.

When this has been run before it gave the following outputs:

/# PART 1


der1

CP001868.2	324403	<	327666

NC_013967.1	329564	<	731435

CP001868.2	737775	<	765441

To check these simply enter the following coordinates in the IGV viewer above:

CP001868.2:324403-327666

NC_013967.1:329564-731435

CP001868.2:737775-765441